# Package downloads and Imports

In [2]:
!pip install -q youtube-dl katna opencv-python
!apt install python3-opencv -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-opencv is already the newest version (3.2.0+dfsg-4ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.


In [1]:
import cv2
import numpy as np
import os
import shutil

In [2]:
from torchvision import transforms # To perform all the transforms on our data
from torchvision import datasets #Used to load the data from the folders
from torch.utils.data import DataLoader
from torchvision import models
import torch
import torch.nn as nn
import torch.optim as optim
import time
from PIL import Image
import matplotlib.pyplot as plt

# Video Download and Keyframe Extraction

In [3]:
!youtube-dl -f 134 --recode-video mp4 https://www.youtube.com/watch?v=2ievbVhn-YU

[youtube] 2ievbVhn-YU: Downloading webpage
[download] Dirt Shark - 2015 Monster Girl Bikini BTS Shoot-2ievbVhn-YU.mp4 has already been downloaded
[download] 100% of 7.09MiB
[ffmpeg] Not converting video file Dirt Shark - 2015 Monster Girl Bikini BTS Shoot-2ievbVhn-YU.mp4 - already is in target format mp4


In [4]:
!ls | grep .mp4

Big Buck Bunny-YE7VzlLtp-4.mp4
Dirt Shark - 2015 Monster Girl Bikini BTS Shoot-2ievbVhn-YU.mp4


In [34]:
def videotoimages(imgname):
    prev_frame = None
    k = 0
    file_path = os.getcwd()
    try:
        print("Folder Created")
        os.mkdir("data/"+imgname)
    except FileExistsError:
        print("Deleting contents as Folder Already Exist")
        shutil.rmtree("data/"+imgname)
        os.mkdir("data/"+imgname)
    video = cv2.VideoCapture(imgname)
    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print("Length of the Video:",length)

    def mse(imageA, imageB):
        err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
        err /= float(imageA.shape[0] * imageA.shape[1])
        return err
    
    while True:
        check,current_frame = video.read()
        if check== False:
            break
        if prev_frame is None:
            prev_frame = current_frame
            continue
        prev_frame = cv2.dilate(prev_frame,None,iterations =0)
        current_frame = cv2.dilate(current_frame,None,iterations =0)
        error = mse(current_frame,prev_frame)
        if(error > 2500 ): # and similarity
            k = k+1
            file_name = file_path +'/data/'+ str(imgname)+"/"+ str(k) +'.jpg'
            resized = cv2.resize(prev_frame, (224,224), interpolation = cv2.INTER_AREA)
            cv2.imwrite(file_name,resized) # Assuming the current frame will be checked with something else
        prev_frame = current_frame
    video.release()
    cv2.destroyAllWindows()
    print("Total Key Frames Extracted:",k)
    return k

In [35]:
k = videotoimages("Dirt Shark - 2015 Monster Girl Bikini BTS Shoot-2ievbVhn-YU.mp4")

Folder Created
Deleting contents as Folder Already Exist
Length of the Video: 4495
Total Key Frames Extracted: 46


# Image Classification

In [59]:
image_classes = ['Drawing','Hentai','Neutral','Pornography','Sexually Provocative']

In [14]:
image_transforms = {
    'train':transforms.Compose([
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test':transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid':transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [85]:
# Predict fucnction that returns the appopirate class index
def predict(model, test_image_name):
    transform = image_transforms['test']
    test_image = Image.open(test_image_name)
    # plt.imshow(test_image)
    test_image_tensor = transform(test_image)
    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        ps = torch.exp(out)
        topk, topclass = ps.topk(1, dim=1)
        index=topclass.cpu().numpy()[0][0]
        # print("Output class :  ", classes[index])
        return index

In [103]:
model = torch.load("data/model_batch-8.pt") # Load Model

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [50]:
# Predict every keyframe extracted
temp=[]
for i in range(1,k):
    temp.append(predict(model,"data/Dirt Shark - 2015 Monster Girl Bikini BTS Shoot-2ievbVhn-YU.mp4/"+str(i)+".jpg"))

In [102]:
from collections import Counter
import math 
total_occurance = Counter(temp).items()
print ("\n{:<21} {:<1} {:<15} {:<1} {:<15}".format("Class Name","|", "Occurance","|", "Percentage"))
print("---------------------------------------------------------")
for key in total_occurance:
    b=(key[1]/k)*100
    print ("{:<21} {:<1} {:<15} {:<1} {:<15}".format(image_classes[key[0]],"|",key[1],"|", str(round(b,2))+" %"))


Class Name            | Occurance       | Percentage     
---------------------------------------------------------
Sexually Provocative  | 17              | 36.96 %        
Neutral               | 26              | 56.52 %        
Drawing               | 2               | 4.35 %         
